In [89]:
import pandas as pd
import sqlite3
from sqlite3 import Error


In [90]:
## Lets build functions that gives percentages based off of a moneyline
def get_pct_from_moneyline(moneyline):
    
    moneyline = int(moneyline)
    
    ## If the moneyline is negative
    if moneyline < 0:
        
        pct = (-1 * moneyline) / ((-1 * moneyline) + 100)

    ## If the moneyline is positive
    elif moneyline > 0:
        
        pct = 100 / (moneyline + 100)

    pct = 100 * round(pct, 4)
    return pct

In [91]:
## We get our data from the data warehouse
data_warehouse = r"C:\Users\arbis\Projects\nate_silver_gets_me_money\database\dk_538_games_dw.db" 
con = None
try:
    con = sqlite3. connect(data_warehouse)
except Error as e:
    print(e)

game_dw = pd.read_sql_query("SELECT * FROM game_dw WHERE date(game_date) >= date('now');", con)

con.close()

game_dw.head()

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk
0,2021-09-08,Mariners,Astros,0.38,0.62,150,-170
1,2021-09-08,Giants,Rockies,0.58,0.42,-145,125
2,2021-09-08,Rangers,Diamondbacks,0.44,0.56,140,-160
3,2021-09-08,Twins,Indians,0.42,0.58,115,-135
4,2021-09-08,Tigers,Pirates,0.50,0.50,100,-120


In [92]:
## then apply our odds function to our draftkings moneyline to get our draftkings raw probability percentage
game_dw["raw_home_odds_dk"] = game_dw["home_moneyline_dk"].apply(get_pct_from_moneyline)
game_dw["raw_away_odds_dk"] = game_dw["away_moneyline_dk"].apply(get_pct_from_moneyline)

## sports book odds add up to 100 + the padding percentage, or their estimated profit margin
game_dw["dk_total_padding_val"] = game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"] - 100

## This padding percentage is (THEORETICALLY) added equally on both sides, or at least in very close amounts
## We rescale the raw percentages back to a 100% total scale to get our final percentage
game_dw["home_odds_dk"] = game_dw["raw_home_odds_dk"] / (game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"])
game_dw["home_odds_dk"] = round(game_dw["home_odds_dk"], 4)

game_dw["away_odds_dk"] = game_dw["raw_away_odds_dk"] / (game_dw["raw_home_odds_dk"] + game_dw["raw_away_odds_dk"])
game_dw["away_odds_dk"] = round(game_dw["away_odds_dk"], 4)

## Now lets look at differences between 538's probabilities and draftkings
game_dw['diff_pct_home'] = game_dw["home_pct_538"] - game_dw["home_odds_dk"]
game_dw['diff_pct_away'] = game_dw["away_pct_538"] - game_dw["away_odds_dk"]

game_dw.head(5)

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk,raw_home_odds_dk,raw_away_odds_dk,dk_total_padding_val,home_odds_dk,away_odds_dk,diff_pct_home,diff_pct_away
0,2021-09-08,Mariners,Astros,0.38,0.62,150,-170,62.96,40.00,2.96,0.6115,0.3885,0.0085,-0.0085
1,2021-09-08,Giants,Rockies,0.58,0.42,-145,125,44.44,59.18,3.62,0.4289,0.5711,-0.0089,0.0089
2,2021-09-08,Rangers,Diamondbacks,0.44,0.56,140,-160,61.54,41.67,3.21,0.5963,0.4037,-0.0363,0.0363
3,2021-09-08,Twins,Indians,0.42,0.58,115,-135,57.45,46.51,3.96,0.5526,0.4474,0.0274,-0.0274
4,2021-09-08,Tigers,Pirates,0.50,0.50,100,-120,54.55,50.00,4.55,0.5218,0.4782,-0.0218,0.0218


In [93]:
def bet_outcome(bet, moneyline):
    
    moneyline = int(moneyline)
    
    ## if the moneyline is negative (betting favortie)
    if moneyline < 0:
        
        outcome = (bet  
                   100) / abs(moneyline)
        
    ## if the moneyline is positive (underdog)
    if moneyline > 0:
        
        outcome = (bet * moneyline) / 100

    outcome = bet + round(outcome , 2)
        
    return outcome

In [94]:
## We get our winnings columns by applying our bet_ouctome function to the original moneyline
game_dw["winnings_on_1_dollar_home"] = game_dw["home_moneyline_dk"].apply(lambda ml: bet_outcome(1, ml))
game_dw["winnings_on_1_dollar_away"] = game_dw["away_moneyline_dk"].apply(lambda ml: bet_outcome(1, ml))

## And get our expected outcome by multipling the winnings by the probability
game_dw["expected_outcome_home_538"] = (game_dw["winnings_on_1_dollar_home"] * game_dw["home_pct_538"]) - 1
game_dw["expected_outcome_away_538"] = (game_dw["winnings_on_1_dollar_away"] * game_dw["away_pct_538"]) - 1


game_dw[["game_date","home_team","away_team","expected_outcome_home_538","expected_outcome_away_538", "away_moneyline_dk"]].head(15)

,game_date,home_team,away_team,expected_outcome_home_538,expected_outcome_away_538,away_moneyline_dk
0,2021-09-08,Astros,Mariners,-0.0142,-0.0500,150
1,2021-09-08,Rockies,Giants,-0.0550,-0.0198,-145
2,2021-09-08,Diamondbacks,Rangers,-0.0928,0.0560,140
3,2021-09-08,Indians,Twins,0.0092,-0.0970,115
4,2021-09-08,Pirates,Tigers,-0.0850,0.0000,100
5,2021-09-08,Marlins,Mets,-0.0641,-0.0667,-120
6,2021-09-08,Yankees,Blue Jays,-0.0442,-0.0340,110
7,2021-09-08,Orioles,Royals,-0.0550,-0.0265,-130
8,2021-09-08,Red Sox,Rays,-0.1023,0.0123,-110
9,2021-09-08,Braves,Nationals,-0.1120,0.1000,175


In [95]:
game_dw.head()

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk,raw_home_odds_dk,raw_away_odds_dk,dk_total_padding_val,home_odds_dk,away_odds_dk,diff_pct_home,diff_pct_away,winnings_on_1_dollar_home,winnings_on_1_dollar_away,expected_outcome_home_538,expected_outcome_away_538
0,2021-09-08,Mariners,Astros,0.38,0.62,150,-170,62.96,40.00,2.96,0.6115,0.3885,0.0085,-0.0085,1.59,2.50,-0.0142,-0.0500
1,2021-09-08,Giants,Rockies,0.58,0.42,-145,125,44.44,59.18,3.62,0.4289,0.5711,-0.0089,0.0089,2.25,1.69,-0.0550,-0.0198
2,2021-09-08,Rangers,Diamondbacks,0.44,0.56,140,-160,61.54,41.67,3.21,0.5963,0.4037,-0.0363,0.0363,1.62,2.40,-0.0928,0.0560
3,2021-09-08,Twins,Indians,0.42,0.58,115,-135,57.45,46.51,3.96,0.5526,0.4474,0.0274,-0.0274,1.74,2.15,0.0092,-0.0970
4,2021-09-08,Tigers,Pirates,0.50,0.50,100,-120,54.55,50.00,4.55,0.5218,0.4782,-0.0218,0.0218,1.83,2.00,-0.0850,0.0000


Couple of things to notice:
 - For some games, both sides have a positive expected value
 - If you add the draft king percentabes together, you will get a value higher than 100
 - This is because draft kings pads their percentages, the generally add 1-2% to each outcome, this becomes their profit margin

To get account for this percentage padding, we adjust our expected outcome by subtracting half of the total percentage padding to each 538 prediction, and recalculate our expected value

In [96]:
game_dw["adj_expected_outcome_away_538"] = game_dw["winnings_on_1_dollar_away"] * (game_dw["away_pct_538"] - game_dw['dk_total_padding_val'] / 100 / 2) - 1
game_dw["adj_expected_outcome_home_538"] = game_dw["winnings_on_1_dollar_home"] * (game_dw["home_pct_538"] - game_dw['dk_total_padding_val'] / 100 / 2) - 1

game_dw.head()

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk,raw_home_odds_dk,raw_away_odds_dk,dk_total_padding_val,home_odds_dk,away_odds_dk,diff_pct_home,diff_pct_away,winnings_on_1_dollar_home,winnings_on_1_dollar_away,expected_outcome_home_538,expected_outcome_away_538,adj_expected_outcome_away_538,adj_expected_outcome_home_538
0,2021-09-08,Mariners,Astros,0.38,0.62,150,-170,62.96,40.00,2.96,0.6115,0.3885,0.0085,-0.0085,1.59,2.50,-0.0142,-0.0500,-0.087000,-0.037732
1,2021-09-08,Giants,Rockies,0.58,0.42,-145,125,44.44,59.18,3.62,0.4289,0.5711,-0.0089,0.0089,2.25,1.69,-0.0550,-0.0198,-0.050389,-0.095725
2,2021-09-08,Rangers,Diamondbacks,0.44,0.56,140,-160,61.54,41.67,3.21,0.5963,0.4037,-0.0363,0.0363,1.62,2.40,-0.0928,0.0560,0.017480,-0.118801
3,2021-09-08,Twins,Indians,0.42,0.58,115,-135,57.45,46.51,3.96,0.5526,0.4474,0.0274,-0.0274,1.74,2.15,0.0092,-0.0970,-0.139570,-0.025252
4,2021-09-08,Tigers,Pirates,0.50,0.50,100,-120,54.55,50.00,4.55,0.5218,0.4782,-0.0218,0.0218,1.83,2.00,-0.0850,0.0000,-0.045500,-0.126632


In [97]:
game_dw.loc[(game_dw['adj_expected_outcome_away_538'] > 0) | (game_dw['adj_expected_outcome_home_538'] > 0)][["game_date", "away_team", "home_team", "away_pct_538", "home_pct_538", "away_moneyline_dk", "home_moneyline_dk", "adj_expected_outcome_away_538", "adj_expected_outcome_home_538"]]

,game_date,away_team,home_team,away_pct_538,home_pct_538,away_moneyline_dk,home_moneyline_dk,adj_expected_outcome_away_538,adj_expected_outcome_home_538
2,2021-09-08,Rangers,Diamondbacks,0.44,0.56,140,-160,0.017480,-0.118801
9,2021-09-08,Nationals,Braves,0.40,0.60,175,-210,0.043625,-0.142340
12,2021-09-08,Dodgers,Cardinals,0.50,0.50,-130,110,-0.151639,0.006530
13,2021-09-08,Angels,Padres,0.36,0.64,230,-290,0.111110,-0.173622


In [98]:
game_dw.to_csv('sample_calculated_data.csv', index = False)